In [22]:
import pandas as pd

In [24]:
avond = pd.read_csv(r"demand_for_wijken_avond.csv")
night = pd.read_csv(r"demand_for_wijken_nacht.csv")
ochtend = pd.read_csv(r"demand_for_wijken_ochtend.csv")
middag = pd.read_csv(r"demand_for_wijken_middag.csv")

# Create column that explains which daypart the frame is from, useful for arcGIS layer
avond['Part of day'] = ['Evening' for i in range(len(avond))]
night['Part of day'] = ['Night' for i in range(len(night))]
ochtend['Part of day'] = ['Morning' for i in range(len(ochtend))]
middag['Part of day'] = ['Afternoon' for i in range(len(middag))]

dfs = [ochtend,middag,avond,night]
demand=pd.DataFrame(columns=list(night.columns))

# Create 1 frame
for df in dfs:
    demand = pd.concat([demand,df])

candidates = pd.read_csv('parkingspots_revised.csv')
times = pd.read_csv('travelling_times.csv')
demands = pd.read_csv('wijken_incidents_updated.csv')
firestations = pd.read_csv('firestations_updated.csv')
ttfs = pd.read_csv('travelling_times_firestations2.csv')


# Dataframe with travelling times from candidate locations to demand locations and from fire station
# locations to demand locations
ttfs=ttfs.rename(columns={'origin_coords':'fire_station_coords','travel_time':'travel_time_fs'})
times = pd.merge(times,ttfs)

# Make frame usable to match names during optimization
candidates['space_coords'] = [f'{lat}_{lon}' for lat,lon in candidates[['latitude','longitude']].values]
firestations['space_coords'] = [f'{lat}_{lon}' for lat,lon in firestations[['latitude','longitude']].values]
demand['demand_coords'] = [f'{lat}_{lon}' for lat,lon in demand[['LAT','LNG']].values]

# Create frame with demands and travelling times
distances = pd.merge(demand,times)
distances = distances.drop(['LAT','LNG','wijkcode','wijknaam'],axis=1)



In [25]:
"""
This function gives a penalty to the demand when a demand location is outside the maximum minute range of a 
candidate location. This function is also used to penalize on how many fire stations have the demand location
within the maximum minute range.
"""

def penalty(distance,max_range,demand):
    if distance >= max_range and distance < max_range*2:
        return demand - 1
    elif distance >= max_range and distance < max_range*3:
        return demand - 2
    elif distance >= max_range:
        return demand - 3
    else:
        return demand


In [26]:
"""
This is the optimization function. It uses demands and distances to try and maximize demand, with use of
penalties. The x (fire_trucks) candidate locations that cover the most demand are the candidate locations
(parking spots) that the function returns as output.
"""

def optimize_locations(demand,distances,candidates,daypart,fire_trucks,max_range):
    
    # Because the input frames use seconds instead of minutes
    max_range=max_range*60
    
    # Only look at given part of day
    distances = distances[distances['Part of day'] == daypart]
    distances = distances.reset_index()
    
    demand = demand[demand['Part of day'] == daypart]
    demand = demand.reset_index()

    # All candidate location names
    cnames = distances['origin_coords'].unique().tolist()
    # All demand location names
    dnames = list(demand['demand_coords'].values)

    num_demands = len(dnames)
    candidates_len = len(cnames)

    # Create a dictionary with candidate locations as keys and the demand they are able to cover
    # as values
    demand_opt = {}
    for i in range(candidates_len):
        demands = 0
        for j in range(num_demands):
            distance = distances[(distances['demand_coords'] == dnames[j]) & 
            (distances['origin_coords'] == cnames[i])]['travel_time'].unique()
            
            distance_station = distances[(distances['demand_coords'] == dnames[j]) & 
            (distances['origin_coords'] == cnames[i])]
            distance_station = distance_station[distance_station['travel_time_fs'] < max_range]

            # Make sure there is a value present
            if len(distance) == 1:
                distance = int(distance[0])
                dem = int(demand[demand['demand_coords']==dnames[j]]['demand'].values[0])
                
                # Base penalty on how much outside the maximum minute range
                # And penalize how on how many fire stations are within the range
                demands += penalty(distance,max_range,dem) + penalty(len(distance_station),3,dem) 
                
        demand_opt[cnames[i]] = demands
        
    # Create dataframe with fire_trucks best results, which can be transfered to a csv if needed
    df = pd.DataFrame(pd.Series({k: v 
                    for k, v in sorted(demand_opt.items(), key=lambda item: item[1], reverse=True)[:fire_trucks]}),
                     columns=['Demand'])
        
    # Convert back to minutes
    max_range=int(max_range/60)
    
    df['Latitude'] = [i.split('_')[0] for i in df.index]
    df['Longitude'] = [i.split('_')[1] for i in df.index]
    df['Parking Space'] = [candidates[candidates['space_coords'] == name]['parking space'].values[0] 
                           for name in df.index]
    df['Placed fire trucks'] = [fire_trucks for i in range(len(df))]
    df['Minute range'] = [max_range for i in range(len(df))]
    df['Part of day'] = [daypart for i in range(len(df))]
    
    df = df.reset_index()
    df = df.drop(['index','Demand'],axis=1)
    
    return df
    

In [29]:
"""
Loop over all possibilities of variables, chosen to use for the dashboard, and concatenate every output 
from the optimization fucntion into one dataframe.
"""

## EVEN DUBBEL CHECKEN ALLES???!
trucks=[2,3,4,5,6]
mins = [2,3,4,5]
part = ['Morning','Afternoon','Evening','Night']

df=pd.DataFrame(columns=['Latitude','Longitude','Parking Space','Placed fire trucks','Minute range','Part of day'])

for n in trucks:
    for m in mins:
        for p in part:
            t=optimize_locations(demand,distances,candidates,p,fire_trucks=n,max_range=m)
            df = pd.concat([df,t])
        

df=df.reset_index()
df=df.drop('index',axis=1)
df

KeyboardInterrupt: 

In [23]:
#df.to_csv('CompleteDF.csv')